In [1]:
import sys
!{sys.executable} -m pip install pydicom -q

In [48]:
import torchvision
import torch
import pydicom as dicom

import numpy as np
import pandas as pd
import rnsa

from sklearn.model_selection import train_test_split
from sklearn import metrics

In [20]:
%load_ext autoreload
%autoreload 1

In [126]:
train_img_dir = '/home/tstrebel/assets/rnsa-pneumonia/train-images'
annotations_file_path = '/home/tstrebel/assets/rnsa-pneumonia/stage_2_train_labels.csv.zip'

label_df = pd.read_csv(annotations_file_path).groupby('patientId').first().reset_index()

X_train, X_test = train_test_split(label_df, test_size=.2, stratify=label_df.Target, random_state=99)
X_val, X_test, = train_test_split(X_test, test_size=.5, stratify=X_test.Target, random_state=99)
train_ix, val_ix, test_ix = X_train.index.tolist(), X_val.index.tolist(), (X_test.index.tolist())
del(X_train)
del(X_val)
del(X_test)
print('train {:,} - validate {:,} - test {:,}'.format(len(train_ix), len(val_ix), len(test_ix)))

train 21,347 - validate 2,668 - test 2,669


In [125]:
mean = [0.5]
std = [0.225]

train_transform = torchvision.transforms.Compose([
    torchvision.transforms.RandomHorizontalFlip(),
    # torchvision.transforms.RandomRotation((-5, 5)),
    torchvision.transforms.Resize(512),
    torchvision.transforms.CenterCrop(448),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean, std),
])

val_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize(512),
    torchvision.transforms.CenterCrop(448),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean, std),
])
    
train_dataset = rnsa.RNSADataset(train_img_dir, annotations_file_path, train_ix, train_transform)
val_dataset = rnsa.RNSADataset(train_img_dir, annotations_file_path, val_ix, val_transform)

In [119]:
model = rnsa.Densenet121(torchvision.models.densenet121(weights='DEFAULT'))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

for param in model.features.parameters():
    param.requires_grad = False

optimizer = torch.optim.SGD(model.classifier.parameters(), lr=1e-3, momentum=.9, weight_decay=1e-4)
criterion = torch.nn.BCEWithLogitsLoss()
lr_scheduler = rnsa.LRScheduler(optimizer)

In [136]:
rnsa.train_model(model,  
                 train_dataset, 
                 val_dataset,
                 optimizer, 
                 criterion, 
                 batch_size=8,
                 num_epochs=1)

epoch 1/1
----------


KeyboardInterrupt: 

device(type='cpu')